# Explanatory Data Analysis (EDA)

In this notebook, I am going to analyze the input data `room.txt` and guide you through my thinking process to find the solution to this problem.

## First approch that comes to my mind

1. Split the big string into multiple strings: one for each room
2. For each string, extract the name of the corresponding room
3. For each string, count the number of each chair
4. Gather all the results to calculate the "total"